# 0_sync_repo: Repository Sync & Clean (Run When Instructors Instruct)

This notebook performs repository synchronization and deep cleaning only. Run it whenever instructors announce an update. Do NOT re-run heavy environment diagnostics here; those now live exclusively in `0_diagnostics.ipynb` and should normally be executed only once at the course start (unless you change your environment).

Key actions provided:
- Safe preview (`dry_run`) of pending git changes
- Forced hard reset to the teaching branch
- Optional deep clean removing untracked & ignored files plus custom stale patterns
- Quick status check cell for routine verification

Destructive warning: A forced reset with deep clean will delete uncommitted work and ignored cache/output folders. Copy personal experiments elsewhere first (e.g. `~/own_model_files/`).

In [ ]:
# Sync repository (single cell)
from importlib import reload
import sys
from pathlib import Path
from datetime import datetime

# Ensure support src path
support_src = Path('SUPPORT_REPO/src').resolve()
if not support_src.exists():
    raise RuntimeError("Expected SUPPORT_REPO/src at project root but it was not found.")
if str(support_src) not in sys.path:
    sys.path.insert(0, str(support_src))

import repo_sync  # type: ignore
reload(repo_sync)

# User-adjustable flags (edit then run this cell)
FORCE_RESET = False          # destructive if True and changes exist
ALLOW_LOCAL_RESET = False    # allow destructive reset off JupyterHub
DRY_RUN = True               # start with True to preview actions
TARGET_REMOTE = 'origin'
TARGET_BRANCH = 'course_2025'
DEEP_CLEAN = False           # set True to remove untracked + optional ignored + extras
INCLUDE_IGNORED = False      # only applies if DEEP_CLEAN True
EXTRA_PATTERNS = [           # relative glob patterns for stale dirs/files (examples)
    # '__pycache__',
    # '.ipynb_checkpoints',
    # '*/_modflow_diag',
]

print(f"[SYNC] Started {datetime.utcnow().isoformat()}Z")
res = repo_sync.sync_repository(
    force_reset=FORCE_RESET,
    allow_local_reset=ALLOW_LOCAL_RESET,
    dry_run=DRY_RUN,
    target_remote=TARGET_REMOTE,
    target_branch=TARGET_BRANCH,
    deep_clean=DEEP_CLEAN,
    deep_clean_include_ignored=INCLUDE_IGNORED,
    deep_clean_extra=EXTRA_PATTERNS if EXTRA_PATTERNS else None,
    verbose=True,
)
try:
    repo_sync.print_sync_summary(res)
except Exception:
    print(res)
print("[SYNC] Done. Set DRY_RUN=False and re-run to apply actions.")

## When to Run What

- Use this sync notebook: whenever instructors announce an update or if you suspect your local copy drifted.
- Use `0_diagnostics.ipynb`: only once at course start, or after you modify your Python environment (install/remove packages) or experience unexplained errors.

If a sync introduces new dependencies, re-run the diagnostics notebook afterwards to confirm readiness.

In [ ]:
# Quick status check (always safe) - does not modify repo
status = repo_sync.sync_repository(dry_run=True, verbose=False)
print(status)